In [1]:
import boto3, re
import sagemaker
from sagemaker import get_execution_role
import os
import numpy as np
import tensorflow as tf
import utils
import pandas as pd
import zipfile
import datetime
import time
import tfrecord_conversion
from sagemaker.predictor import csv_serializer, json_deserializer,RealTimePredictor

sagemaker_session = sagemaker.Session()
role = get_execution_role()
bucket='tensorflow-rul'
prefix='RUL'

In [2]:
# source_data_location = 's3://sagemaker-solutions-us-west-2/Predictive-maintenance-using-machine-learning/data'

# #local data folder
# data_folder = 'data'

# !aws s3 cp --recursive $source_data_location $data_folder


# with zipfile.ZipFile(os.path.join(data_folder, 'CMAPSSData.zip'), "r") as zip_ref:
#     zip_ref.extractall(data_folder)

In [28]:
# train_df, test_df, columns = utils.preprocess_data(data_folder)
# test_set = pd.DataFrame()
# for i in train_df:
#     train_set=pd.DataFrame(i)

# for i in test_df:
#     test_set=pd.DataFrame(i)
# test_set.to_csv('test_set.csv')
# print(test_set.shape[1])


train_set = pd.read_csv('newtrain.csv', index_col = 0)

test_set = pd.read_csv('newtest.csv', index_col = 0)

dataset = tfrecord_conversion.data_preprocess(train_set, test_set)
print(dataset["train_dict"]["features"].shape)
print(dataset["train_dict"]["labels"].shape)
print(dataset["test_dict"]["features"].shape)
print(dataset["test_dict"]["labels"].shape)

from numpy import asarray
from numpy import save
# define data

# save to npy file

save('train_feat.npy',dataset["train_dict"]["features"])
save('train_label.npy',dataset["train_dict"]["labels"])
save('test_feat.npy',dataset["test_dict"]["features"])
save('test_label.npy',dataset["test_dict"]["labels"])


(20626, 5, 24)
(20626,)
(13091, 5, 24)
(13091,)


In [4]:
!mkdir 'tfrecord_data'



tfrecord_conversion.convert_to(dataset["train_dict"], 'train', 'tfrecord_data')
tfrecord_conversion.convert_to(dataset["test_dict"], 'test', 'tfrecord_data')

mkdir: cannot create directory ‘tfrecord_data’: File exists
Writing tfrecord_data/train.tfrecords

Writing tfrecord_data/test.tfrecords


In [5]:
inputs = sagemaker_session.upload_data(path='tfrecord_data',bucket=bucket, key_prefix='data/RUL_Tfrecord')

In [6]:
!cat dnn_regressor.py

import os
import numpy as np
import tensorflow as tf
from tensorflow.python.estimator.model_fn import ModeKeys as Modes

INPUT_TENSOR_NAME = 'inputs'
SIGNATURE_NAME = 'predictions'
LEARNING_RATE = 0.001

from tensorflow.python.estimator.model_fn import ModeKeys as Modes

def model_fn(features, labels, mode, params):
    # Input Layer


#     input_layer = tf.reshape(features[INPUT_TENSOR_NAME],[-1,26])

#     dense1 = tf.layers.dense(inputs=input_layer, units=128, activation=tf.nn.relu)
# #    drop1 = tf.layers.dropout(dense1, rate=0.5)
#     dense2 = tf.layers.dense(inputs=dense1, units=64, activation=tf.nn.relu)
# #    drop2 = tf.layers.dropout(dense2, rate=0.5)
#     dense3 = tf.layers.dense(inputs=dense2, units=32, activation=tf.nn.relu)
# #     drop3 = tf.layers.dropout(dense3, rate=0.5)
#     dense4 = tf.layers.dense(inputs=dense3, units=16, activation=tf.nn.relu)
# #     drop4 = tf.layers.dropout(dense3, rate=0.5)
#     logits = tf.layers.dense(inputs=dense4, units=1, activation

In [7]:
from sagemaker.tensorflow import TensorFlow
output_path = "s3://tensorflow-rul/RUL"

RUL_estimator = TensorFlow(entry_point='dnn_regressor.py',
                             role=role,
                             framework_version='1.11.0',
                             output_path = output_path,
                             training_steps=3000, 
                             evaluation_steps=100,
                             #early_stopping="true",
                             #patience="5",
                             train_instance_count=1,
                             train_instance_type='ml.c4.xlarge')

RUL_estimator.fit(inputs)

2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-20 11:10:28 Starting - Starting the training job...
2020-10-20 11:10:30 Starting - Launching requested ML instances......
2020-10-20 11:11:32 Starting - Preparing the instances for training...
2020-10-20 11:12:28 Downloading - Downloading input data
2020-10-20 11:12:28 Training - Downloading the training image...
2020-10-20 11:12:47 Training - Training image download completed. Training in progress.2020-10-20 11:12:47,457 INFO - root - running container entrypoint
2020-10-20 11:12:47,458 INFO - root - starting train task
2020-10-20 11:12:47,471 INFO - container_support.training - Training starting
2020-10-20 11:12:50,059 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2020-10-20 11:12:50,059 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2020-10-20 11:12:50,059 INFO - tf_container - ---------------------------------------------------------
2020-10-20 11:12:50,059 INFO

2020-10-20 11:13:05,812 INFO - tensorflow - Evaluation [90/100]
2020-10-20 11:13:05,821 INFO - tensorflow - Evaluation [100/100]
2020-10-20 11:13:05,849 INFO - tensorflow - Finished evaluation at 2020-10-20-11:13:05
2020-10-20 11:13:05,849 INFO - tensorflow - Saving dict for global step 3000: global_step = 3000, loss = 2066.2896, rmse = 45.45646
2020-10-20 11:13:05.861791: E tensorflow/core/platform/s3/aws_logging.cc:60] No response body. Response code: 404
2020-10-20 11:13:05.861829: W tensorflow/core/platform/s3/aws_logging.cc:57] If the signature check failed. This could be because of a time skew. Attempting to adjust the signer.
2020-10-20 11:13:05.916036: E tensorflow/core/platform/s3/aws_logging.cc:60] No response body. Response code: 404
2020-10-20 11:13:05.916087: W tensorflow/core/platform/s3/aws_logging.cc:57] If the signature check failed. This could be because of a time skew. Attempting to adjust the signer.
2020-10-20 11:13:05.952008: E tensorflow/core/platform/s3/aws_logg

In [8]:
# output_path = '/'.join(RUL_estimator.output_path.split('/')[:-1])

# print(output_path)
# # 

# optimized_estimator = RUL_estimator.compile_model(target_instance_family='rasp3b',
#                               output_path=output_path,
#                               input_shape= {'data':[1,5,24]},  # Batch size 1, 3 channels, 224x224 Images.
#                               framework='tensorflow', framework_version='1.11.0')


In [9]:
RUL_predictor = RUL_estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!

In [10]:
RUL_predictor.content_type = 'text/csv'
RUL_predictor.serializer = csv_serializer
RUL_predictor.deserializer = json_deserializer

In [11]:
test=test_set
test = test.drop(columns='RUL')
test.to_csv('trial.csv')

In [12]:
print(test.shape)

(13096, 26)


In [13]:
test_dataset = pd.read_csv("trial.csv", index_col= 0)

test_feature_dataset = test_dataset[['id', 'cycle','setting1', 'setting2', 'setting3', 's1', 's2', 's3','s4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14','s15', 's16', 's17', 's18', 's19', 's20', 's21'
]]
#print(test_feature_dataset)
test_features = np.array(test_feature_dataset.values).astype('float32')

In [14]:
# for tf in test_features:
#     prediction=RUL_predictor.predict(tf)
#     print(prediction['outputs']['Pred_RUL']['floatVal'][0])

In [15]:



# x = []
# y1 = []
# # y2 = [] 
# id_data = []
# id_data = np.array(test_set[test['id']==1]['RUL'])
# print(id_data.shape)
# for i in range(len(id_data)):x.append(i)
# print(x)
# import matplotlib.pyplot as plt 


# for tf in x:
    
# #     print(test_features[tf])
# #     print(tf)
#     prediction = RUL_predictor.predict(tfrecord_conversion.lstm_preprocess(test_set,5))
# #     x.append(tf)
#     y1.append(prediction['outputs']['Pred_RUL']['floatVal'][0])
# #     y2.append(test_set['RUL'].loc[tf])
# #print(test_set[test['id']==2])
# #     print(id_data[tf],prediction['outputs']['Pred_RUL']['floatVal'][0]  )

# print(len(y1))
# plt.plot(x, y1, color='k', label='Predicted RUL')
# plt.plot(x, id_data, color='b', label='Actual RUL')

# plt.xlabel('Cycles')
# plt.ylabel('RUL')

# print("RUL", id_data[0], max(y1))

# # plt.legend()
# # plt.savefig('testplot.png')

In [16]:
# import matplotlib.pyplot as plt 
# x = []
# y1 = []
# y2 = [] 
# id_no = 1
# id_data = []
# id_data = np.array(test_set[test_set['id']==id_no]['RUL'])
# print(id_data)
# for i in range(len(id_data)):x.append(i)
# print(x)
# test_features = np.array(test[test['id']==id_no])
# for tf in x:
#     prediction = RUL_predictor.predict(test_features[tf])
#     y1.append(prediction['outputs']['Pred_RUL']['floatVal'][0])

# print(y1)

# plt.plot(x, y1, color='k', label='Predicted RUL')
# plt.plot(x, id_data, color='b', label='Actual RUL')

# plt.xlabel('Cycles')
# plt.ylabel('RUL')
# plt.legend()
# plt.savefig('testplot.png')

In [17]:
# optimized_predictor = optimized_estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

In [18]:
# endpoint = 'insert name of your endpoint here'
# payload = test_dataset
# predictor = RealTimePredictor(endpoint=endpoint, content_type='text/csv')
# inference_response = predictor.predict(data=payload)
# print (inference_response)

In [19]:
# optimized_predictor.content_type = 'text/csv'
# optimized_predictor.serializer = csv_serializer
# optimized_predictor.deserializer = json_deserializer

In [27]:
import matplotlib.pyplot as plt 
x = []
y1 = []
y2 = [] 

testdata = tfrecord_conversion.lstm_preprocess(test_set,5)
# print(testdata.shape)
pred = RUL_predictor.predict(testdata)
print(pred)
# for tf in range(10):
#     print(tf, testdata[tf])
# #     prediction = RUL_predictor.predict(testdata[tf])
# #     x.append(tf)
# #     y1.append(prediction['outputs']['Pred_RUL']['floatVal'][0])
# #     y2.append(test_set['RUL'].loc[tf])
# plt.plot(x, y1, color='k', label='Predicted RUL')
# plt.plot(x, y2, color='b', label='Actual RUL')

# plt.xlabel('Cycles')
# plt.ylabel('RUL')
# plt.legend()
# plt.savefig('testplot.png')


{'outputs': {'Pred_RUL': {'dtype': 'DT_FLOAT', 'floatVal': [99.37246704101562], 'tensorShape': {'dim': [{'size': '1'}, {'size': '1'}]}}}, 'modelSpec': {'version': '1603192386', 'name': 'generic_model', 'signatureName': 'serving_default'}}
